# LSTM Classifier
***

In [171]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.random import set_seed
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
import talos
from talos.utils import lr_normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras import metrics
import keras.backend as K
from functools import partial


Checking whether tensorflow is running on GPU:

In [172]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  1
/device:GPU:0


Importing final dataset:

In [173]:
df = pd.read_csv('datasets/final_db.csv', index_col=0)
df.sort_index(inplace=True)
df.head(3)

,target,mean_sent,STOCH_slowk,STOCH_slowd,MACD,MACD_hist,CCI_5,CCI_10,CCI_21,CCI_50,MOM_5,MOM_10,MOM_21,RSI_5,RSI_10,RSI_21,RSI_50
Day,,,,,,,,,,,,,,,,,
2015-01-01,0.0,0.011820,67.198744,57.422805,-12.540186,4.426599,-76.487357,-58.930944,-57.963081,-8.626979,-28.059998,-48.588989,-80.842987,30.94113,30.495281,36.523877,43.526454
2015-01-02,1.0,0.009464,67.198744,57.422805,-12.540186,4.426599,-76.487357,-58.930944,-57.963081,-8.626979,-28.059998,-48.588989,-80.842987,30.94113,30.495281,36.523877,43.526454
2015-01-03,0.0,0.009646,67.198744,57.422805,-12.540186,4.426599,-76.487357,-58.930944,-57.963081,-8.626979,-28.059998,-48.588989,-80.842987,30.94113,30.495281,36.523877,43.526454


---
### Model & hyperparameter tuning with Talos

In [240]:
# Function to get class weights:
def cwts(data):
    c0, c1 = np.bincount(data['target'])
    #making the weights inversely proportional to the amount of observations:
    w0, w1 = (1/c0)*(len(data))/2, (1/c1)*(len(data))/2
    return {0: w0, 1:w1}

# Defining Recall, Precision, Specificity and F1 metrics:
#(https://medium.com/analytics-vidhya/custom-metrics-for-keras-tensorflow-ae7036654e05)
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras


def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    return tn / (tn + fp + K.epsilon())


def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

#========================================================================================#
# This function keeps the initial learning rate for the first ten epochs
# and decreases it exponentially after that.
def scheduler(epoch, lr):
    return np.clip(lr * tf.math.exp(-0.1*epoch), 0.000001, 0.001) #limited to these learning rates

#========================================================================================#
#LSTM ARCHITECTURE
def lstm_model(x_train, y_train, x_val, y_val, params):

    tf.keras.backend.clear_session()
    #This function defines the LSTM model
    win_length = params['window']
    batch_size = params['batch_size']
    num_features= x_train.shape[1]
    train_generator = TimeseriesGenerator(scaledtrain, y_train, length= win_length, sampling_rate = 1, batch_size= batch_size)
    test_generator = TimeseriesGenerator(scaledtest, y_val, length= win_length, sampling_rate = 1, batch_size= batch_size)
    #==================================================================#
    # MODEL ARCHITECTURE
    model = Sequential()
    
    # #Input and first LSTM layer:
    # model.add(LSTM(params['neurons'], activation=params['activation'],input_shape = (win_length, num_features), return_sequences = True))
    # model.add(Dropout(params['dropout']))
    # # Dropout layer after LSTM layer
    # # model.add(Dropout(params['dropout']))

    # # Another stacked LSTM layer, to allow for complexity
    # model.add(LSTM(params['neurons'], activation=params['activation'],return_sequences=True))
    # # Dropout layer after LSTM layer
    # model.add(Dropout(params['dropout']))

    
    # # Third layer to receive all information
    # model.add(LSTM(params['neurons'], activation=params['activation'], return_sequences=False))

    # # DEnse layer to yield all info to output, reducing dimension
    # # if params['activation'] == 'leakyrelu':
    # #     model.add(Dense(params['dense_neurons'], activation = tf.keras.layers.LeakyReLU(alpha=0.01)))#output layer
    # # else:
    # #     model.add(Dense(params['dense_neurons'], activation = params['activation']))#output layer

    # model.add(LSTM(128, input_shape = (win_length, num_features), return_sequences = True))
    # model.add(LeakyReLU(alpha=0.5))
    # model.add(LSTM(128, return_sequences=True))
    # model.add(LeakyReLU(alpha=0.5))
    # model.add(Dropout(0.3))
    # model.add(LSTM(64, return_sequences=False))
    # model.add(Dropout(0.3))
    # model.add(Dense(1, activation='sigmoid'))

    
    
    
    #Input and first LSTM layer:
    model.add(LSTM(params['neurons'],input_shape = (win_length, num_features), return_sequences = True))
    
    # Dropout layer after LSTM layer
    model.add(Dropout(params['dropout']))

    # Another stacked LSTM layer, to allow for complexity
    model.add(LSTM(params['neurons'], return_sequences=True))
    # Dropout layer after LSTM layer
    model.add(Dropout(params['dropout']))

    
    # Third layer to receive all information
    model.add(LSTM(params['neurons'], return_sequences=False))

    # DEnse layer to yield all info to output, reducing dimension
    if params['activation'] == 'leakyrelu':
        model.add(Dense(params['dense_neurons'], activation = tf.keras.layers.LeakyReLU(alpha=0.01)))#output layer
    else:
        model.add(Dense(params['dense_neurons'], activation = params['activation']))#output layer
    
    # Output layer (class =0 or 1)
    model.add(Dense(1, activation = 'sigmoid'))#output layer

    #==================================================================#
    # Defining Early Stopping to avoid overfitting (after 3 attempts)
    early_stopping = EarlyStopping(monitor='loss',patience = 5, mode='min', restore_best_weights=True)

    # Using exponential decay:
    
    callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

    # Compiling
    # (lr=lr_normalizer(params['lr'], params['optimizer']))
    # callbacks = [early_stopping, callbacklr],
    model.compile(loss='binary_crossentropy', 
                    optimizer = params['optimizer'], 
                    metrics=['accuracy',precision,recall,f1,specificity,])
    
    history = model.fit(train_generator,
                        validation_data=test_generator, # validation sample with features
                        epochs = params['epochs'], # validation sample with labels
                        shuffle=False, callbacks = [early_stopping, callbacklr],
                        class_weight = class_weight, # imbalanced data
                        verbose=1
                        )

    return(history, model)

To facilitate computation we scale the features.

In [242]:
# We scale the data so hasten the LSTM's conversion
scaler = MinMaxScaler() 


# Features x Target

features = df.iloc[:,1:]
target = df.iloc[:,0]

# Shuffle is set to False because it is a timeseries, and the order matters.
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0, shuffle = False)

# Scaling
scaledtrain = scaler.fit_transform(x_train)
scaledtest = scaler.transform(x_test)

We then define the class weights (imbalanced):

In [243]:
class_weight = cwts(df)
class_weight

{0: 1.1066666666666667, 1: 0.9120879120879121}

In [244]:
# Defining parameters considered in the hyperparameter tuning
params = {'lr': [1e-2, 1e-3, 1e-4],
        'neurons': [32, 64, 128], 
        'dropout': [0.3, 0.5, 0.8],
        'batch_size': [32, 64, 256], 
        'epochs': [50,100],
        'optimizer': ['Adam','Nadam'],
        'activation': ['leakyrelu','relu'],
        'dense_neurons': [4, 8, 16],
        'window': [21,50,200],
        }

# Scanning for best hyperparameters with Talos
scan = talos.Scan(x = x_train, y = y_train, x_val = x_test, y_val = y_test, params = params, model = lstm_model,experiment_name = 'lstm_model_val')

Epoch 1/50
45/45 [==============================] - 6s 42ms/step - loss: 0.6925 - accuracy: 0.4434 - precision: 0.0949 - recall: 0.0661 - f1: 0.0559 - specificity: 0.9327 - val_loss: 0.6933 - val_accuracy: 0.4783 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 0.0010
Epoch 2/50
45/45 [==============================] - 1s 20ms/step - loss: 0.6923 - accuracy: 0.4441 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1: 0.0000e+00 - specificity: 1.0000 - val_loss: 0.6932 - val_accuracy: 0.4783 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 9.0484e-04
Epoch 3/50
45/45 [==============================] - 1s 20ms/step - loss: 0.6922 - accuracy: 0.4503 - precision: 0.2023 - recall: 0.2493 - f1: 0.1806 - specificity: 0.7552 - val_loss: 0.6931 - val_accuracy: 0.5217 - val_precision: 0.5201 - val_recall: 1.0000 - val_f1: 0.6782 - val_specificity: 0.0000e+00 - lr: 7.4082e-04
Epoch 4/50

Epoch 1/50
45/45 [==============================] - 6s 50ms/step - loss: 0.6919 - accuracy: 0.4858 - precision: 0.3761 - recall: 0.5802 - f1: 0.4183 - specificity: 0.4160 - val_loss: 0.6930 - val_accuracy: 0.5222 - val_precision: 0.5214 - val_recall: 1.0000 - val_f1: 0.6790 - val_specificity: 0.0000e+00 - lr: 0.0010
Epoch 2/50
45/45 [==============================] - 1s 24ms/step - loss: 0.6918 - accuracy: 0.5582 - precision: 0.5569 - recall: 1.0000 - f1: 0.7098 - specificity: 0.0000e+00 - val_loss: 0.6929 - val_accuracy: 0.5222 - val_precision: 0.5214 - val_recall: 1.0000 - val_f1: 0.6790 - val_specificity: 0.0000e+00 - lr: 9.0484e-04
Epoch 3/50
45/45 [==============================] - 1s 27ms/step - loss: 0.6918 - accuracy: 0.5582 - precision: 0.5569 - recall: 1.0000 - f1: 0.7098 - specificity: 0.0000e+00 - val_loss: 0.6928 - val_accuracy: 0.5222 - val_precision: 0.5214 - val_recall: 1.0000 - val_f1: 0.6790 - val_specificity: 0.0000e+00 - lr: 7.4082e-04
Epoch 4/50
45/45 [============

Epoch 1/50
40/40 [==============================] - 7s 84ms/step - loss: 0.6919 - accuracy: 0.5603 - precision: 0.5692 - recall: 0.9429 - f1: 0.6990 - specificity: 0.0660 - val_loss: 0.6967 - val_accuracy: 0.4337 - val_precision: 0.4427 - val_recall: 1.0000 - val_f1: 0.6112 - val_specificity: 0.0000e+00 - lr: 0.0010
Epoch 2/50
40/40 [==============================] - 2s 59ms/step - loss: 0.6915 - accuracy: 0.5087 - precision: 0.5088 - recall: 0.7149 - f1: 0.5513 - specificity: 0.2645 - val_loss: 0.6953 - val_accuracy: 0.4337 - val_precision: 0.4427 - val_recall: 1.0000 - val_f1: 0.6112 - val_specificity: 0.0000e+00 - lr: 9.0484e-04
Epoch 3/50
40/40 [==============================] - 2s 59ms/step - loss: 0.6907 - accuracy: 0.5595 - precision: 0.5619 - recall: 0.9848 - f1: 0.7111 - specificity: 0.0078 - val_loss: 0.6968 - val_accuracy: 0.4337 - val_precision: 0.4427 - val_recall: 1.0000 - val_f1: 0.6112 - val_specificity: 0.0000e+00 - lr: 7.4082e-04
Epoch 4/50
40/40 [====================

Epoch 1/50
45/45 [==============================] - 5s 42ms/step - loss: 0.6929 - accuracy: 0.5094 - precision: 0.3364 - recall: 0.4456 - f1: 0.3453 - specificity: 0.5692 - val_loss: 0.6938 - val_accuracy: 0.4783 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 0.0010
Epoch 2/50
45/45 [==============================] - 1s 20ms/step - loss: 0.6940 - accuracy: 0.4934 - precision: 0.2952 - recall: 0.4640 - f1: 0.3372 - specificity: 0.5377 - val_loss: 0.6922 - val_accuracy: 0.5217 - val_precision: 0.5201 - val_recall: 1.0000 - val_f1: 0.6782 - val_specificity: 0.0000e+00 - lr: 9.0484e-04
Epoch 3/50
45/45 [==============================] - 1s 20ms/step - loss: 0.6924 - accuracy: 0.5518 - precision: 0.5556 - recall: 0.9852 - f1: 0.7040 - specificity: 0.0137 - val_loss: 0.6929 - val_accuracy: 0.5217 - val_precision: 0.5201 - val_recall: 1.0000 - val_f1: 0.6782 - val_specificity: 0.0000e+00 - lr: 7.4082e-04
Epoch 4/50
45/45 [============

Epoch 1/50
45/45 [==============================] - 6s 43ms/step - loss: 0.6932 - accuracy: 0.4979 - precision: 0.3903 - recall: 0.6475 - f1: 0.4745 - specificity: 0.3263 - val_loss: 0.6925 - val_accuracy: 0.5222 - val_precision: 0.5214 - val_recall: 1.0000 - val_f1: 0.6790 - val_specificity: 0.0000e+00 - lr: 0.0010
Epoch 2/50
45/45 [==============================] - 1s 16ms/step - loss: 0.6939 - accuracy: 0.5028 - precision: 0.3926 - recall: 0.5843 - f1: 0.4308 - specificity: 0.4099 - val_loss: 0.6931 - val_accuracy: 0.5222 - val_precision: 0.5214 - val_recall: 1.0000 - val_f1: 0.6790 - val_specificity: 0.0000e+00 - lr: 9.0484e-04
Epoch 3/50
45/45 [==============================] - 1s 17ms/step - loss: 0.6924 - accuracy: 0.5191 - precision: 0.5659 - recall: 0.6693 - f1: 0.5130 - specificity: 0.3491 - val_loss: 0.6924 - val_accuracy: 0.5222 - val_precision: 0.5214 - val_recall: 1.0000 - val_f1: 0.6790 - val_specificity: 0.0000e+00 - lr: 7.4082e-04
Epoch 4/50
45/45 [====================

Epoch 1/50
40/40 [==============================] - 7s 85ms/step - loss: 0.6906 - accuracy: 0.5579 - precision: 0.5218 - recall: 0.8035 - f1: 0.6003 - specificity: 0.1988 - val_loss: 0.6872 - val_accuracy: 0.5663 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 0.0010
Epoch 2/50
40/40 [==============================] - 2s 57ms/step - loss: 0.6930 - accuracy: 0.5175 - precision: 0.3888 - recall: 0.5779 - f1: 0.4369 - specificity: 0.4172 - val_loss: 0.6916 - val_accuracy: 0.5663 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 9.0484e-04
Epoch 3/50
40/40 [==============================] - 2s 57ms/step - loss: 0.6930 - accuracy: 0.4833 - precision: 0.3557 - recall: 0.5046 - f1: 0.3730 - specificity: 0.4975 - val_loss: 0.6926 - val_accuracy: 0.5663 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 7.4082e-04
Epoch 4/50
40/

Epoch 1/50
45/45 [==============================] - 6s 45ms/step - loss: 0.6937 - accuracy: 0.4837 - precision: 0.3053 - recall: 0.4617 - f1: 0.3400 - specificity: 0.5487 - val_loss: 0.6925 - val_accuracy: 0.5217 - val_precision: 0.5201 - val_recall: 1.0000 - val_f1: 0.6782 - val_specificity: 0.0000e+00 - lr: 0.0010
Epoch 2/50
 7/45 [===>..........................] - ETA: 0s - loss: 0.7019 - accuracy: 0.4955 - precision: 0.4929 - recall: 1.0000 - f1: 0.6583 - specificity: 0.0075    